In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
# tf.VERSION
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

# global varuables
w = h = 9

C:\Users\Abe-r\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Abe-r\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Abe-r\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Abe-r\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi

### Reading of Images

In [2]:
#
# Reads band images and index images
#
def read_img(site, _s):
    data = []

    for i in range(1, 8):
        if _s == 'B':
            filename = "input/"+ site +"/B" + str(i) + "_250_B.tif"
        else:
            filename = "input/"+ site +"/B" + str(i) + "_250.tif"
            
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        data.append(img)

    index = ["NDVI.tif", "NDWI.tif", "NDBI.tif"]
    for i in index:
        filename = "input/"+ site +"/" + i
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        data.append(img)

#     cv2.imshow("Image", data[9])
#     cv2.waitKey(0)
    
    return data

i_img = read_img("Landsat8-A", 'A')
test_img = read_img("Landsat8-B", 'B')

### Crop Region of Interest
- extract 9x9 pixels from point(x, y)

In [3]:
# Training

def crop_train(_img, _c, site):
    _class = ""
    if(_c == 0):
        _class = 'Vegetation'
    elif(_c == 1):
        _class = 'Built-up'
    else:
        _class = 'Water'
        
    _pts = pd.read_excel('input/'+ site +'.xlsx', sheet_name = _class)

    # iterate over rows with iterrows()
    for index, row in _pts.iterrows():
        x = row['X'] - 4
        y = row['Y'] - 4
        
        for img in _img:        
            crop_img = img[y:y+h, x:x+w]
        
    #         if(len(crop_img[5]) < 9):
    #         print(row['X'], row['Y'], len(crop_img), len(crop_img[0]), len(crop_img[1]))
            X_tr_list.append(crop_img)
        y_tr.append(_c)
        

In [4]:
# Testing

def crop_test(_img):
    _pts = pd.read_excel('input/site-b.xlsx')

    for index, row in _pts.iterrows():
        x = row['X'] - 4
        y = row['Y'] - 4
        
        if(row['Class'] == 255):
            _c = 0
        elif(row['Class'] == 128):
            _c = 1
        else:
            _c = 2
        
        for img in _img:        
            crop_img = img[y:y+h, x:x+w]
            X_te_list.append(crop_img)
        y_te.append(_c)
        

### Training and Testing Dataset

In [5]:
# TRAINING SET SITE A ------------------------------------------------------------------
X_tr_list = []
y_tr = []

crop_train(i_img, 0, 'site-a')
crop_train(i_img, 1, 'site-a')
crop_train(i_img, 2, 'site-a')

print("Training...")
print(len(X_tr_list))
print(len(y_tr))

# TRAINING SET SITE B ------------------------------------------------------------------
X_te_list = []
y_te = []

crop_test(i_img)
print("Testing...")
print(len(X_te_list))
print(len(y_te))


Training...
2000
200
Testing...
2000
200


### Reshaping Training and Testing Dataset

In [6]:

X_tr = []
X_tr = np.array(X_tr_list).reshape(-1, 10, 9, 9)

print(X_tr[0].shape)
print(X_tr[0, 9], y_tr[0])


X_te = []
X_te = np.array(X_te_list).reshape(-1, 10, 9, 9)

print(X_te[0].shape)
print(X_te[0, 9], y_te[0])



(10, 9, 9)
[[74 84 68 66 72 67 68 72 80]
 [70 66 66 60 89 64 59 70 66]
 [91 57 62 55 70 62 60 70 68]
 [72 76 61 53 62 53 62 70 65]
 [61 57 62 56 51 50 59 66 66]
 [76 62 62 62 51 53 66 66 70]
 [68 66 56 50 56 50 51 64 68]
 [70 64 56 56 58 51 49 54 60]
 [68 70 56 50 54 54 54 56 68]] 0
(10, 9, 9)
[[ 80  72  89 113 107 107  88  84  76]
 [ 78  80  93  99  90  92  76  78  80]
 [ 80  82  83  78  92  90  74  76  74]
 [ 95  87  82  76  87  91  88  85  70]
 [ 92 103  80  70  74  80 106  74  63]
 [ 96 105 109  74  70 104  90  70  68]
 [ 82  80  97  96  72 101  76  70  76]
 [ 95 110  90  97  86  88  66  64  62]
 [100 103 110  97 101  87  88  76  68]] 2


### First Neural Network: Basic CLassification
- https://www.tensorflow.org/tutorials/keras/basic_classification
#### Setup Layers, Compile Model, and Train Model

In [7]:
# Setup
model_1 = keras.Sequential([
    keras.layers.Flatten(input_shape = X_tr.shape[1:]),
    keras.layers.Dense(1, activation=tf.nn.relu),
    keras.layers.Dense(3, activation=tf.nn.softmax)
])

# Compile
model_1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#Train
model_1.fit(X_tr, y_tr, epochs=5)

Instructions for updating:
Colocations handled automatically by placer.
Epoch 1/5
200/200 [==============================] - 0s 603us/sample - loss: 2.4092 - acc: 0.4650
Epoch 2/5
200/200 [==============================] - 0s 11us/sample - loss: 1.0941 - acc: 0.5950
Epoch 3/5
200/200 [==============================] - 0s 78us/sample - loss: 1.0909 - acc: 0.5950
Epoch 4/5
200/200 [==============================] - 0s 58us/sample - loss: 1.0876 - acc: 0.5950
Epoch 5/5
200/200 [==============================] - 0s 40us/sample - loss: 1.0846 - acc: 0.5950


#### Evaluate Accuracy

In [8]:
test_loss, test_acc = model_1.evaluate(X_te, y_te)

print('Test accuracy:', test_acc)

200/200 [==============================] - 0s 279us/sample - loss: 1.0882 - acc: 0.4050
Test accuracy: 0.405


#### Make Predictions

In [9]:
predictions = model_1.predict(X_te)
predictions[0]

array([0.3329083 , 0.34455064, 0.322541  ], dtype=float32)

In [10]:
np.argmax(predictions[0])

1

In [11]:
y_te[0]

2

## Convolutional Neural Network

In [12]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

#### Convolutional Base

In [13]:
model_2 = models.Sequential()
model_2.add(layers.Conv2D(32, (1, 1), activation='relu', input_shape=X_tr.shape[1:]))
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_2.add(layers.MaxPooling2D((2, 2)))
model_2.add(layers.Conv2D(64, (1, 1), activation='relu'))

In [14]:
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 10, 9, 32)         320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 5, 4, 32)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 2, 64)          18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 64)          4160      
Total params: 22,976
Trainable params: 22,976
Non-trainable params: 0
_________________________________________________________________


#### Dense Layer

In [15]:
model_2.add(layers.Flatten())
model_2.add(layers.Dense(1, activation='relu'))
model_2.add(layers.Dense(3, activation='softmax'))

In [16]:
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 10, 9, 32)         320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 5, 4, 32)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 2, 64)          18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 64)          4160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
__________

#### Compile Model

In [17]:
model_2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_2.fit(X_tr, y_tr, epochs=5)

Epoch 1/5
200/200 [==============================] - 0s 982us/sample - loss: 2.1644 - acc: 0.5050
Epoch 2/5
200/200 [==============================] - 0s 194us/sample - loss: 1.0942 - acc: 0.5950
Epoch 3/5
200/200 [==============================] - 0s 130us/sample - loss: 1.0912 - acc: 0.5950
Epoch 4/5
200/200 [==============================] - 0s 135us/sample - loss: 1.0882 - acc: 0.5950
Epoch 5/5
200/200 [==============================] - 0s 125us/sample - loss: 1.0852 - acc: 0.5950


#### Evaluate Model

In [18]:
test_loss, test_acc = model_2.evaluate(X_te, y_te)

200/200 [==============================] - 0s 334us/sample - loss: 1.0906 - acc: 0.4050
